# 카트폴 균형 잡기 위한 RL 훈련

이 노트북은 [AI for Beginners Curriculum](http://aka.ms/ai-beginners)의 일부입니다. [공식 PyTorch 튜토리얼](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)과 [이 카트폴 PyTorch 구현](https://github.com/yc930401/Actor-Critic-pytorch)에서 영감을 받았습니다.

이 예제에서는 RL을 사용하여 수평 축에서 좌우로 움직일 수 있는 카트 위에 막대를 균형 있게 세우는 모델을 훈련시킬 것입니다. 우리는 [OpenAI Gym](https://www.gymlibrary.ml/) 환경을 사용하여 막대를 시뮬레이션할 것입니다.

> **Note**: 이 강의의 코드는 로컬(예: Visual Studio Code)에서 실행할 수 있으며, 이 경우 시뮬레이션이 새 창에서 열립니다. 온라인으로 코드를 실행할 경우, [여기](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7)에 설명된 대로 코드를 약간 수정해야 할 수도 있습니다.

우리는 Gym이 설치되어 있는지 확인하는 것부터 시작하겠습니다:


In [ ]:
import sys
!{sys.executable} -m pip install gym

이제 CartPole 환경을 생성하고 이를 어떻게 작동시키는지 살펴보겠습니다. 환경은 다음과 같은 속성을 가지고 있습니다:

* **Action space**는 시뮬레이션의 각 단계에서 수행할 수 있는 가능한 행동들의 집합입니다.  
* **Observation space**는 우리가 관찰할 수 있는 관찰값들의 공간입니다.  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

시뮬레이션이 어떻게 작동하는지 살펴봅시다. 다음 루프는 `env.step`이 종료 플래그 `done`을 반환할 때까지 시뮬레이션을 실행합니다. 우리는 `env.action_space.sample()`을 사용하여 무작위로 행동을 선택할 것이며, 이는 실험이 매우 빠르게 실패할 가능성이 높다는 것을 의미합니다 (CartPole 환경은 CartPole의 속도, 위치 또는 각도가 특정 한계를 벗어날 때 종료됩니다).

> 시뮬레이션은 새 창에서 열립니다. 코드를 여러 번 실행하여 동작을 확인할 수 있습니다.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

관찰값은 4개의 숫자로 구성되어 있습니다. 이 숫자는 다음을 나타냅니다:
- 카트의 위치
- 카트의 속도
- 막대의 각도
- 막대의 회전 속도

`rew`는 매 단계마다 받는 보상입니다. CartPole 환경에서는 시뮬레이션 단계마다 1점을 받으며, 목표는 총 보상을 최대화하는 것입니다. 즉, CartPole이 넘어지지 않고 균형을 유지할 수 있는 시간을 늘리는 것이 목표입니다.

강화 학습에서는 **정책** $\pi$를 훈련시키는 것이 목표입니다. 이 정책은 각 상태 $s$에 대해 어떤 행동 $a$를 취해야 하는지 알려줍니다. 즉, 본질적으로 $a = \pi(s)$입니다.

확률적 해결 방법을 원한다면, 정책을 각 행동에 대한 확률 집합을 반환하는 것으로 생각할 수 있습니다. 즉, $\pi(a|s)$는 상태 $s$에서 행동 $a$를 취해야 할 확률을 의미합니다.

## 정책 경사법

가장 간단한 강화 학습 알고리즘인 **정책 경사법**에서는 신경망을 훈련시켜 다음 행동을 예측하도록 합니다.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

우리는 여러 실험을 실행하고 각 실행 후 네트워크를 업데이트하여 네트워크를 훈련할 것입니다. 실험을 실행하고 결과(소위 **추적**) - 모든 상태, 행동(및 권장 확률), 보상을 반환하는 함수를 정의해 봅시다:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

훈련되지 않은 네트워크로 하나의 에피소드를 실행하고 총 보상(즉, 에피소드 길이)이 매우 낮다는 것을 관찰할 수 있습니다:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

정책 경사 알고리즘의 까다로운 측면 중 하나는 **할인된 보상**을 사용하는 것입니다. 아이디어는 게임의 각 단계에서 총 보상의 벡터를 계산하고, 이 과정에서 일부 계수 $gamma$를 사용하여 초기 보상을 할인하는 것입니다. 또한 결과 벡터를 정규화하는데, 이는 이를 훈련에 영향을 미치는 가중치로 사용할 것이기 때문입니다.


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

이제 실제 훈련을 시작해봅시다! 우리는 300개의 에피소드를 실행할 것이며, 각 에피소드에서 다음을 수행합니다:

1. 실험을 실행하고 추적 데이터를 수집합니다.
1. 수행된 행동과 예측된 확률 간의 차이(`gradients`)를 계산합니다. 차이가 적을수록 올바른 행동을 취했을 가능성이 높아집니다.
1. 할인된 보상을 계산하고 이를 `gradients`에 곱합니다. 이렇게 하면 높은 보상을 받은 단계가 낮은 보상을 받은 단계보다 최종 결과에 더 큰 영향을 미치게 됩니다.
1. 신경망의 예상 목표 행동은 실행 중 예측된 확률과 계산된 `gradients`에서 부분적으로 가져옵니다. 우리는 `alpha` 매개변수를 사용하여 `gradients`와 보상이 어느 정도로 고려될지를 결정합니다. 이것은 강화 알고리즘의 *학습률*이라고 불립니다.
1. 마지막으로, 상태와 예상 행동을 기반으로 네트워크를 훈련시키고 과정을 반복합니다.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

이제 렌더링을 통해 에피소드를 실행하여 결과를 확인해 봅시다:


In [ ]:
_ = run_episode(render=True)

이제 막대가 꽤 잘 균형을 잡을 수 있다는 것을 확인할 수 있습니다!

## 액터-크리틱 모델

액터-크리틱 모델은 정책 기울기(policy gradients)를 더욱 발전시킨 형태로, 정책과 예상 보상을 동시에 학습하는 신경망을 구축하는 방식입니다. 이 신경망은 두 가지 출력을 가지며(혹은 두 개의 별도 네트워크로 볼 수도 있습니다):
* **액터(Actor)**는 정책 기울기 모델에서처럼 상태 확률 분포를 제공하여 어떤 행동을 취할지 추천합니다.
* **크리틱(Critic)**은 해당 행동들로부터 얻을 수 있는 보상을 예측합니다. 주어진 상태에서 미래에 예상되는 총 보상을 반환합니다.

이러한 모델을 정의해 봅시다:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

우리는 `discounted_rewards` 및 `run_episode` 함수를 약간 수정해야 합니다:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


이제 주요 학습 루프를 실행하겠습니다. 적절한 손실 함수를 계산하고 네트워크 매개변수를 업데이트하여 수동 네트워크 학습 프로세스를 사용할 것입니다.


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## 주요 내용

이 데모에서는 두 가지 강화 학습 알고리즘을 살펴보았습니다: 간단한 정책 경사법과 더 정교한 액터-크리틱 방법. 이러한 알고리즘은 상태, 행동, 보상이라는 추상적인 개념을 기반으로 작동하므로 매우 다양한 환경에 적용할 수 있습니다.

강화 학습은 최종 보상을 관찰하는 것만으로 문제를 해결하기 위한 최적의 전략을 학습할 수 있게 해줍니다. 라벨이 지정된 데이터셋이 필요하지 않다는 점은 모델을 최적화하기 위해 시뮬레이션을 여러 번 반복할 수 있다는 장점을 제공합니다. 하지만 강화 학습에는 여전히 많은 도전 과제가 있으며, 이 흥미로운 AI 분야에 더 집중하기로 결정한다면 이러한 과제들을 배우게 될 것입니다.



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 신뢰할 수 있는 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
